# Integrated Labs Assignment: Simple Blockchain Implementation

In [1]:
import hashlib
import datetime

In [2]:
class Block:
    def __init__(self, previous_block_hash, data, timestamp):
        self.previous_block_hash = previous_block_hash
        self.data = data
        self.timestamp = timestamp
        self.hash = self.get_hash()

    def create_genesis_block():
        return Block("0", "0", datetime.datetime.now())

    def get_hash(self):
        header_bin = (str(self.previous_block_hash) + str(self.data) + str(self.timestamp)).encode()

        inner_hash = hashlib.sha256(header_bin).hexdigest().encode()
        outer_hash = hashlib.sha256(inner_hash).hexdigest()
        return outer_hash

In [6]:
b1 = Block.create_genesis_block()
b1.hash

'c64bb18f4f8fc9ed3ad5c39ebb2eafe880dc5c5dd562a5314ec6748bc54d968b'

In [5]:
block_chain = [Block.create_genesis_block()]

print("The Genesis Block has been created")
print("Hash: %s" % block_chain[-1].hash)

The Genesis Block has been created
Hash: a6f94a30a5ce182f0490c911ef47538177938794b3cea4e677c49a86d3f162f9


In [9]:
#Adding 5 blocks

num_blocks_to_add = 5

for i in range(1, num_blocks_to_add+1):
    block_chain.append(Block(block_chain[-1].hash, "DATA!", datetime.datetime.now()))

    print("Block #%d has been created." % i)
    print("Block #%d hash: %s" % (i, block_chain[i].hash))

Block #1 has been created.
Block #1 hash: b3230d8527059af1923af3e7cf5be47dad5c37d615338b6db43381382ddb5513
Block #2 has been created.
Block #2 hash: 875359d809033f502785f656a13f291561cba2fd23806c8775bbc576041fd0af
Block #3 has been created.
Block #3 hash: 945cb6e3ed9075a6893bb6e50be8b03ae9e9338ddcfb9b7bdb372e4d00c51d4f
Block #4 has been created.
Block #4 hash: 015b9d97f64bc9cd68f125390bc7c51fc936046569c8cdf09f5c35af55eab001
Block #5 has been created.
Block #5 hash: b8e0a04b8296018531378585df6c07d753e47645c2b9710775eee16eb175848c
